In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [26]:
data_fake=pd.read_csv(r"D:\detection\Data sets\telugu_false.csv")
data_true=pd.read_csv(r"D:\detection\Data sets\telugu_real.csv")

In [27]:
data_fake.shape,data_true.shape

((54, 2), (56, 2))

In [28]:
print(data_fake.isnull().sum())
print(data_true.isnull().sum())

text     0
class    0
dtype: int64
text     0
class    0
dtype: int64


In [29]:
print(data_fake.head(), "\n")
print(data_true.head())

                                                text  class
0  చంద్రయాన్-3 మిషన్ విజయవంతమైన తర్వాత, చంద్రుడిప...      0
1  ప్లాస్టిక్ గుడ్లు, ప్లాస్టిక్ బియ్యం తెలుగు రా...      0
2  కొత్తగా ప్రవేశపెట్టిన ₹2000 నోట్లలో GPS చిప్ ఉ...      0
3  కొవిడ్-19 వ్యాక్సిన్ తీసుకున్నవారు జంతువులుగా ...      0
4  కొన్ని సందర్భాల్లో, మాజీ ముఖ్యమంత్రులు నందమూరి...      0 

                                                text  class
0  పవన్ కళ్యాణ్ ఫోటో మార్ఫింగ్‌పై కేసులు: ఆంధ్రప్...      1
1  కేటీఆర్ హైకోర్టును ఆశ్రయించారు: తెలంగాణ రాష్ట్...      1
2  మంత్రి శ్రీధర్‌బాబు వ్యాఖ్యలు: తెలంగాణలో కాంగ్...      1
3  PNB రుణాలపై వడ్డీ రేట్లు తగ్గింపు: పంజాబ్ నేషన...      1
4  పాల శీతలీకరణ కేంద్రం సీజ్‌పై పాడి రైతుల నిరసన:...      1


In [30]:
data = pd.concat([data_fake,data_true],axis=0)

In [31]:
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

In [32]:
print(data['class'].value_counts())

class
1    56
0    54
Name: count, dtype: int64


In [33]:
X = data["text"]  # News content
y = data["class"] 

In [34]:
import re
import string
import pandas as pd
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS  # For stopword removal

# Load Telugu stopwords (You can add more words manually)
telugu_stopwords = set(["అంటే", "కాని", "ఉంది", "తర్వాత", "అతను", "ఆమె", "నాకు", "మీరు", "వారు"])  

def clean_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove numbers and special characters
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.translate(str.maketrans("", "", string.punctuation))  # Remove punctuation

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove stopwords (both Telugu & English)
    text = " ".join(word for word in text.split() if word not in telugu_stopwords and word not in ENGLISH_STOP_WORDS)

    return text

# Apply text cleaning to the "text" column
data["cleaned_text"] = data["text"].apply(clean_text)

# Print sample cleaned text
print(data[["text", "cleaned_text"]].head())


                                                text  \
0  టీమిండియా vs బంగ్లాదేశ్: ఛాంపియన్స్ ట్రోఫీలో ట...   
1  చిల్కూర్ బాలాజీ ఆలయంలో రాత్రివేళల్లో స్వయంగా శ...   
2  కొన్ని సందర్భాల్లో, మాజీ ముఖ్యమంత్రులు నందమూరి...   
3  సంచిక' ఫిబ్రవరి సంచిక విడుదల: తెలుగు సాహిత్య వ...   
4  కోబాలి' వెబ్ సిరీస్ విడుదల: రవి ప్రకాష్, శ్యామ...   

                                        cleaned_text  
0  టీమిండియా vs బంగ్లాదేశ్ ఛాంపియన్స్ ట్రోఫీలో టీ...  
1  చిల్కూర్ బాలాజీ ఆలయంలో రాత్రివేళల్లో స్వయంగా శ...  
2  కొన్ని సందర్భాల్లో మాజీ ముఖ్యమంత్రులు నందమూరి ...  
3  సంచిక ఫిబ్రవరి సంచిక విడుదల తెలుగు సాహిత్య వేద...  
4  కోబాలి వెబ్ సిరీస్ విడుదల రవి ప్రకాష్ శ్యామల భ...  


In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [36]:
vectorizer = TfidfVectorizer(max_features=5000)  # Top 5000 words for better performance
X_tfidf = vectorizer.fit_transform(data["cleaned_text"])

In [37]:
y = data["class"]

In [38]:
smote = SMOTE(sampling_strategy="auto", random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_tfidf, y)

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [40]:
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [41]:
y_pred = model.predict(X_test)

In [42]:
print("Model Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Model Accuracy: 0.782608695652174
Classification Report:
               precision    recall  f1-score   support

           0       0.64      1.00      0.78         9
           1       1.00      0.64      0.78        14

    accuracy                           0.78        23
   macro avg       0.82      0.82      0.78        23
weighted avg       0.86      0.78      0.78        23



In [43]:
import pickle

# Save the trained model
pickle.dump(model, open("fake_news_model.pkl", "wb"))

# Save the TF-IDF vectorizer (to transform new text data)
pickle.dump(vectorizer, open("tfidf_vectorizer.pkl", "wb"))

print("Model and vectorizer saved successfully!")


Model and vectorizer saved successfully!


In [44]:
loaded_model = pickle.load(open("fake_news_model.pkl", "rb"))
loaded_vectorizer = pickle.load(open("tfidf_vectorizer.pkl", "rb"))


In [45]:
user_text = input("🔎 Enter a news article to check if it's fake or true: ")

In [46]:
user_text_tfidf = loaded_vectorizer.transform([user_text])

In [47]:
prediction = loaded_model.predict(user_text_tfidf)

In [48]:
if prediction[0] == 0:
    print("❌ Fake News Detected!")
else:
    print("✅ This is True News.")

❌ Fake News Detected!
